#### Lesson 2: Sequential Chats and Customer Onboarding

#### Setup

In [1]:
#r "nuget:AutoGen,0.2.0"
#load "util.csx"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using System.Threading;
using OpenAI;
using OpenAI.Chat;

var chatClient = ChatClientProvider.Create("gpt-4o-mini");

Installed Packages AutoGen, 0.2.0

Create the needed agents

In [2]:
var onboardingPersonalInformationAgent = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Onboarding_Personal_Information_Agent",
    systemMessage: """
    You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [3]:
var onboardingTopicPreferenceAgent = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Onboarding_Topic_Preference_Agent",
    systemMessage: """
    You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [4]:
var customerEngagementAgent = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Customer_Engagement_Agent",
    systemMessage: """
    You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [5]:
var summarizer = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Summarizer",
    systemMessage: """
    You are a helpful summarizer agent.
    Your job is to summarize the conversation between the user and the customer service agent.
    Return 'TERMINATE' when you are done.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [6]:
var user = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "User",
    systemMessage: """
    Your name is John and you live in New York.
    You are reaching out to customer service to find out something fun.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

#### Creating Tasks
In python AutoGen, you can use initiate_chats to create and run a sequential of tasks in json object
In dotnet AutoGen, however, that feature is not available, so you need to manually create these tasks using code.

#### Task 1. Onboard customer by gathering name and location
(onboard_personal_information_agent -> user .. (repeat less than two times)) -> summarizer

In [7]:
var greetingMessage = new TextMessage(Role.Assistant, """
    Hello, I'm here to help you get started with our product.
    Could you tell me your name and location?
    """, from: onboardingPersonalInformationAgent.Name);

var conversation = await onboardingPersonalInformationAgent.SendAsync(
    receiver: user,
    [greetingMessage],
    maxRound: 2)
    .ToListAsync();

var summarizePrompt = """
    Return the customer information into as JSON object only: {'name': '', 'location': ''}
    """;

var summary = await summarizer.SendAsync(summarizePrompt, conversation);

from: User
Hi there! My name is John, and I live in New York. I'm excited to get started!

from: Onboarding_Personal_Information_Agent
Great to meet you, John! Thank you for sharing your location. 

TERMINATE

from: Summarizer
{'name': 'John', 'location': 'New York'}



#### Task 2. Gapther customer's preferences on news topics
(onboarding_topic_preference_agent -> user .. (repeat one time)) -> summarizer

In [8]:
var topicPreferenceMessage = new TextMessage(Role.Assistant, """
    Great! Could you tell me what topics you are interested in reading about?
    """, from: onboardingTopicPreferenceAgent.Name);

conversation = await onboardingTopicPreferenceAgent.SendAsync(
    receiver: user,
    [topicPreferenceMessage],
    maxRound: 1)
    .ToListAsync();

// Keep summarizing
summary = await summarizer.SendAsync(chatHistory: new[] { summary }.Concat(conversation));

from: User
Hi there! I'm interested in fun and exciting topics like local events happening in New York, unique restaurants to try, or interesting activities and attractions in the city. Anything that can help me discover something new and enjoyable would be great!

from: Summarizer
The user is looking for recommendations on local events, unique restaurants, and interesting activities and attractions in New York to discover new and enjoyable experiences. 

TERMINATE



#### Task 3. Engage the customer with fun facts, jokes, or interesting stories based on the user's personal information and topic preferences
(user(find fun thing to read) -> customerEngagementAgent .. (repeat 1 time)) -> summarizer

In [9]:
var funFactMessage = new TextMessage(Role.User, """
    Let's find something fun to read.
    """, from: user.Name);

conversation = await user.SendAsync(
    receiver: customerEngagementAgent,
    chatHistory: conversation.Concat([funFactMessage]), // this time, we keep the previous conversation history
    maxRound: 1)
    .ToListAsync();

// Keep summarizing
summary = await summarizer.SendAsync(chatHistory: new[] { summary }.Concat(conversation));

from: Customer_Engagement_Agent
Absolutely! New York City is bursting with excitement and wonderful experiences. Here are a few fun ideas to explore:

### Local Events
1. **Smorgasburg** - This open-air food market in Williamsburg is a foodie paradise. Try unique dishes from local vendors—like Ramen Burgers and artisanal donuts!
2. **Broadway Shows** - Always check for last-minute tickets to catch a dazzling performance. Whether it's a classic like "The Phantom of the Opera" or the latest hit, the energy is unbeatable.
3. **Street Art Tours** - Explore neighborhoods like Bushwick and Harlem, where you can witness some of the most vibrant street art. You can even join a guided tour to learn about the artists and their inspirations!

### Unique Restaurants
1. **The Catbird Seat** - A quirky spot where you can enjoy a tasting menu while seated in an intimate setting. The chef prepares dishes right in front of you!
2. **Ellen's Stardust Diner** - Enjoy classic diner food while being entert